In [26]:
from Lab6 import *
from random import randint
from numpy import array,inf
from time import time
from threading import Thread
import threading
from math import sqrt,ceil
import warnings
warnings.simplefilter("ignore")

In [27]:
ev = threading.Event()
def stop():
    return ev.is_set()

Miller-Rabin test:
- Input: n > 2, an odd integer to be tested for primality;
  -k, a parameter that determines the accuracy of the test
- Output: composite if n is composite, otherwise probably prime
  - write n − 1 as 2s·d with d odd by factoring powers of 2 from n − 1
  - LOOP: repeat k times:
    - pick a randomly in the range [2, n − 1]
    - x ← a d mod n
    - if x = 1 or x = n − 1 then do next LOOP
    - repeat s − 1 times:
      - x ← x 2 mod n
      - if x = 1 then return composite
      - if x = n − 1 then do next LOOP
    - return composite
  - return probably prime

In [28]:
def is_prime_miller_rabin(n, k):
    """
    Input: n > 2, an odd integer to be tested for primality;
       k, a parameter that determines the accuracy of the test
    Output: composite if n is composite, otherwise probably prime
    write n − 1 as 2s·d with d odd by factoring powers of 2 from n − 1
    LOOP: repeat k times:
       pick a randomly in the range [2, n − 1]
       x ← a d mod n
       if x = 1 or x = n − 1 then do next LOOP
       repeat s − 1 times:
          x ← x 2 mod n
          if x = 1 then return composite
          if x = n − 1 then do next LOOP
       return composite
    return probably prime
    """
    if not n&1 or n==1:
        return False
    s=1
    d=n-1
    while not d&1:
        s*=2
        d//=2
    for i in range(k):
        a = randint(2,n-1)
        x = pow(a,d,n)
        if x==1 or x==n-1:
            continue
        for j in range(s-1):
            if stop(): 
                return
            x = pow(x,2,n)
            if x==1:
                return False
            if x==n-1:
                break
        if x==n-1:
            continue
        return False
    return True

compareExecutionTime:
- A - a primality test function
- B - a primality test function
- reps - how many times to repeat the test
- timelimit - maximum execution time
- kwargs - arguments that are used by methods A and B
  - Variables:
  - res - execution result
  - r1 - result of A(kwargs) or None
  - r2 = result of B(kwargs) or None
- each time test is executed in a separate Thread, in order to make it possible to stop the method invocation after a fixed time.
- a mean execution time and the standard deviation is computed after executing all tests.
- return value:
  - mean1 - the mean of the algorithm A execution time
  - std1 - the standard deviation of the algorithm A execution time
  - mean2 - the mean of the algorithm B execution time
  - std2 - the standard deviation of the algorithm B execution time
  - res - execution result (assuming A(kwargs) == B(kwargs))

In [31]:
def compareExecutionTime(A, B, reps=1, timelimit=3, **kwargs):
    t1 = []
    t2 = []
    res=None
    r1=None
    r2=None
    def f1(**kwargs):
        nonlocal r1,t1
        r1=A(**kwargs)
        if not stop():
            t1.append(time() - time1)
    def f2(**kwargs):
        nonlocal r2,t2
        r2=B(**kwargs)
        if not stop():
            t2.append(time() - time2)
    for i in range(reps):
        r1,r2=None,None
        time1 = time()
        
        l=len(t1)
        
        p1 = Thread(target = f1, kwargs = kwargs)
        p1.start()
        p1.join(timeout = timelimit)
        while time()-time1<timelimit and p1.is_alive(): pass
        ev.set()
        while p1.is_alive(): pass
        ev.clear()
        
        if len(t1)==l:
            t1.append(inf)
        
        l=len(t2)
        time2 = time()
        p2 = Thread(target = f2, kwargs = kwargs)
        p2.start()
        p2.join(timeout = timelimit)
        while time()-time2<timelimit and p2.is_alive(): pass
        ev.set()
        while p2.is_alive(): pass
        ev.clear()
        
        if len(t2)==l:
            t2.append(inf)
        
        assert r1 is None or r2 is None or r1==r2
        if res is None:
            res=r1
        r1=None
        r2=None
    t11 = array(t1, dtype=float)
    t21 = array(t2, dtype=float)
    
    return [(t11.mean(), t11.std()), (t21.mean(), t21.std()), res]

Execution time analysis:
- Given two lists of prime and composite numbers
- For each number:
  - Check if a primality test function returns the expected value
  - Test the execution time
  - Print mean and standard deviation of the execution time for each number

In [36]:
prime=[
    3,
    17,
    101,
    1013,
    10099,
    103141,
    1016783,
    10103099,
    107707549,
    10770592813,
    107705911723,
    4547337172376300111955330758342147474062293202868155909489
]
composite=[
    2,
    3*17,
    556,
    800001,
    1007*101,
    103141*1016783,
    245709213523*5299871829441939,
    5059489012128901*1298582838728545,
    4547337172376300111955330758342147474062293202868155909393
]

algorithms=[
    is_prime_miller_rabin
]
args=[
    {"n":1,"k":1}
]
    

def execFor(algo, _n, expected, **kwargs):
    A=algo
    kw=dict()
    print("n =",_n)
    kw['n']=_n
    k=min(10000,max(5,p//1000))
    if "k" in kwargs:
        print("k =",k)
        kw['k']=k
    
    time1,time2,res=compareExecutionTime(A,A,timelimit=1, reps=3,**kw)
    
    print("Mean exec. time of %s:        %.15f" % (A.__name__, time1[0]))
    print("Std. of the exec. time of %s: %.15f" % (A.__name__, time1[1]))
    print("Result (is prime):",res)
    print("\n\n")

for i in range(len(algorithms)):
    algo=algorithms[i]
    kw=args[i]
    print("\n\n\n")
    print(algo.__name__)
    print("\n\n")
    
    for p in prime:
        execFor(algo, p, True,**kw)
    
    for c in composite:
        execFor(algo, c, False,**kw)
    print(end="\n\n\n\n\n")





is_prime_miller_rabin



n = 3
k = 5
Mean exec. time of is_prime_miller_rabin:        0.000590483347575
Std. of the exec. time of is_prime_miller_rabin: 0.000427862642700
Result (is prime): True



n = 17
k = 5
Mean exec. time of is_prime_miller_rabin:        0.000280618667603
Std. of the exec. time of is_prime_miller_rabin: 0.000396854725579
Result (is prime): True



n = 101
k = 5
Mean exec. time of is_prime_miller_rabin:        0.000000000000000
Std. of the exec. time of is_prime_miller_rabin: 0.000000000000000
Result (is prime): True



n = 1013
k = 5
Mean exec. time of is_prime_miller_rabin:        0.000333150227865
Std. of the exec. time of is_prime_miller_rabin: 0.000471145570554
Result (is prime): True



n = 10099
k = 10
Mean exec. time of is_prime_miller_rabin:        0.000000000000000
Std. of the exec. time of is_prime_miller_rabin: 0.000000000000000
Result (is prime): True



n = 103141
k = 103
Mean exec. time of is_prime_miller_rabin:        0.000333229700724
Std. of t

#### $$\textbf{Conclusions}$$
1. Test Millera-Rabina jest probabilistycznym testem pierwszości,
    pozwala z dużym prawdopodobieństwem stwierdzić, czy dana liczba jest liczbą pierwszą.
    Jest znacznie szybszy w porównaniu z nieprobabilistycznym testem pierwszości pierwiastka kwadratowego,
    a dokładność wyników testu można określić i zmienić.
    Ten test wyróżnia się w analizie pierwszorzędności dużych liczb (np. >1e6),
    ze względu na skrócony czas wykonania i zużycie zasobów.
    Wadą jest to, że jego wyniki nie są jednoznaczne z prawdopodobieństwem 100%.
    Niemniej jednak test Millera-Rabina jest powszechnie znany i stosowany. To jest
    uważany za klasyczny test pierwszości.
